# 102: Swissmetro Weighted MNL Mode Choice

In [ ]:
# TEST

import numpy as np
import pandas as pd

import larch

pd.set_option("display.max_columns", 999)
pd.set_option("expand_frame_repr", False)
pd.set_option("display.precision", 3)
np.set_printoptions(precision=12)
larch._doctest_mode_ = True
import larch as lx

In [ ]:
import pandas as pd

import larch as lx

This example is a mode choice model built using the Swissmetro example dataset.
First we create the Dataset and Model objects:

In [ ]:
raw_data = pd.read_csv(lx.example_file("swissmetro.csv.gz")).rename_axis(index="CASEID")
data = lx.Dataset.construct.from_idco(raw_data, alts={1: "Train", 2: "SM", 3: "Car"})
data

The swissmetro example models exclude some observations.  We can use the 
`Dataset.query_cases` method to identify the observations we would like to keep.

In [ ]:
m = lx.Model(data.dc.query_cases("PURPOSE in (1,3) and CHOICE != 0"))

We can attach a title to the model. The title does not affect the calculations
as all; it is merely used in various output report styles.

In [ ]:
m.title = "swissmetro example 02 (weighted logit)"

We need to identify the availability and choice variables.

In [ ]:
m.availability_co_vars = {
    1: "TRAIN_AV * (SP!=0)",
    2: "SM_AV",
    3: "CAR_AV * (SP!=0)",
}
m.choice_co_code = "CHOICE"

This model adds a weighting factor.

In [ ]:
m.weight_co_var = "1.0*(GROUP==2)+1.2*(GROUP==3)"

The swissmetro dataset, as with all Biogeme data, is only in `co` format.

In [ ]:
from larch import P, X

m.utility_co[1] = P("ASC_TRAIN")
m.utility_co[2] = 0
m.utility_co[3] = P("ASC_CAR")
m.utility_co[1] += X("TRAIN_TT") * P("B_TIME")
m.utility_co[2] += X("SM_TT") * P("B_TIME")
m.utility_co[3] += X("CAR_TT") * P("B_TIME")
m.utility_co[1] += X("TRAIN_CO*(GA==0)") * P("B_COST")
m.utility_co[2] += X("SM_CO*(GA==0)") * P("B_COST")
m.utility_co[3] += X("CAR_CO") * P("B_COST")

Larch will find all the parameters in the model, but we'd like to output them in
a rational order.  We can use the ordering method to do this:

In [ ]:
m.ordering = [
    (
        "ASCs",
        "ASC.*",
    ),
    (
        "LOS",
        "B_.*",
    ),
]

In [ ]:
# TEST
from pytest import approx

assert m.loglike() == approx(-7892.111473285806)

We can estimate the models and check the results match up with those given by Biogeme:

In [ ]:
m.set_cap(15)
result = m.maximize_loglike(method="SLSQP")
result

In [ ]:
# TEST
from pytest import approx

assert result.loglike == approx(-5931.557677709527)

In [ ]:
m.calculate_parameter_covariance()
m.parameter_summary()

In [ ]:
# TEST
assert m.pvals == approx(
    [-0.114834339145, -0.756969214206, -0.011197899961, -0.013210667574], rel=1e-2
)
assert m.pstderr == approx(
    [0.040689138346, 0.052839270341, 0.000490421261, 0.000536780123], rel=1e-2
)
assert m.parameter_summary().data["t Stat"].values.astype(float) == approx(
    [-2.82, -14.33, -22.83, -24.61], rel=1e-2
)
assert m.parameter_summary().data["Signif"].values == approx(
    ["**", "***", "***", "***"]
)

Looks good!

In [ ]:
# TEST numba
m.compute_engine = "numba"
m.pvals = 0
m.clear_cache()
assert m.loglike() == approx(-7892.111473285806)
result_n = m.maximize_loglike(method="SLSQP", quiet=True)
assert m.pvals == approx(
    [-0.114834339145, -0.756969214206, -0.011197899961, -0.013210667574], rel=1e-2
)
assert m.pstderr == approx(
    [0.040689138346, 0.052839270341, 0.000490421261, 0.000536780123], rel=1e-3
)
assert m.parameter_summary().data["t Stat"].values.astype(float) == approx(
    [-2.82, -14.33, -22.83, -24.61], rel=1e-2
)
assert m.parameter_summary().data["Signif"].values == approx(
    ["**", "***", "***", "***"]
)